In [2]:
import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df


In [13]:
ids = ['2454100925', '2449863607']
df = read_mongo(db="propertiesdb", collection="apartment", host='localhost', query={"id_vivareal" : {"$in" : ids}})
df

,displayAddressType,amenities,feedsId,contractType,usableAreas,constructionStatus,videoTourLink,listingType,description,title,...,buildings,capacityLimit,status,advertiserContact,price,id_vivareal,medias,link,updated,deliveredAt
0,NEIGHBORHOOD,"[AIR_CONDITIONING, BARBECUE_GRILL, KITCHEN, GO...",AP0020,REAL_ESTATE,[92],ConstructionStatus_NONE,,USED,"APARTAMENTO SOFISTICADO NO PARQUE PRADO, AO LA...","Apartamento com 3 dormitórios à venda, 92 m² p...",...,0,[],ACTIVE,"{'phones': ['1925141288', '19953217715']}",545000,2454100925,[{'url': 'https://resizedimgs.vivareal.com/{ac...,"{'data': {}, 'name': 'Apartamento com 3 Quarto...",2020-04-15 09:45:59.145183,NaN
1,ALL,"[BARBECUE_GRILL, LAUNDRY]",AP000012,REAL_ESTATE,[92],ConstructionStatus_NONE,,USED,"Apartamento a venda o lado do shopping Prado, ...",apartamento - Parque Prado - Campinas,...,0,[],ACTIVE,{'phones': ['19971417711']},580000,2449863607,[{'url': 'https://resizedimgs.vivareal.com/{ac...,"{'data': {}, 'name': 'Apartamento com 3 Quarto...",2020-04-15 09:46:12.448276,2011-04-14T09:17:23.085Z


In [28]:
import os
import shutil

# define the name of the directory to be created
root_path = "/home/dzanardo/github/apartamento/"
IMGFOLDER = '/home/dzanardo/github/apartamento/images/'

for index, row in df.iterrows():
    path = root_path + row['id_vivareal'] + '/'
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    count = 0 
    for m in row['medias']:
        count += 1
        path_old = IMGFOLDER + m['new_url']
        path_new = path + str(count) + '.jpg'
        shutil.copyfile(path_old, path_new)


Creation of the directory /home/dzanardo/github/apartamento/2454100925/ failed
Creation of the directory /home/dzanardo/github/apartamento/2449863607/ failed
